In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [2]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [4]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(10)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."
5,9,"[864335, 990865, 1029743, 9297474, 10457112, 8..."
6,13,"[6534178, 1104146, 829197, 840361, 862070, 884..."
7,14,"[840601, 867293, 933067, 951590, 952408, 96569..."
8,15,"[910439, 1082185, 959076, 1023958, 1082310, 13..."
9,16,"[1062973, 1082185, 13007710]"


In [5]:
test_users = result.shape[0]
new_test_users = len(set(data_test['user_id']) - set(data_train['user_id']))

print('В тестовом дата сете {} юзеров'.format(test_users))
print('В тестовом дата сете {} новых юзеров'.format(new_test_users))

В тестовом дата сете 2042 юзеров
В тестовом дата сете 0 новых юзеров


### Задание 0. Товар 999999
На вебинаре мы использовали товар 999999 - это товар, который купили пользователи, если они купиши товар из top5000. Используя этот товар мы смещяем качество рекомендаций. В какую сторону? Уберите этот товар и сравните с качеством на семинаре.

In [6]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

In [7]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [8]:
data_train_top = data_train.loc[data_train['item_id'].isin(top_5000)]
data_train.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [9]:
new_test_users = len(set(data_test['user_id']) - set(data_train_top['user_id']))

print('В тестовом дата сете {} юзеров'.format(test_users))
print('В тестовом дата сете {} новых юзеров'.format(new_test_users))

В тестовом дата сете 2042 юзеров
В тестовом дата сете 5 новых юзеров


5 пользователей в train'е купили только товары не из топ-5000, но эти пользователи есть в test'е.
Наличие товара 999999 позволило бы не строить отдельную модель для них. Но т.к. они все же есть, можно предложить предсказывать для них самые популярные товары, не вошедшие в топ.

In [10]:
new_test_users = set(data_test['user_id']) - set(data_train_top['user_id'])

In [11]:
user_item_matrix_top = pd.pivot_table(data_train_top, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix_top[user_item_matrix_top > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix_top = user_item_matrix_top.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item_top = csr_matrix(user_item_matrix_top).tocsr()

In [12]:
user_item_matrix_top.shape

(2486, 5000)

In [20]:
data_train_no_top = data_train.loc[~data_train['item_id'].isin(top_5000)]

popular_no_top = data_train_no_top.groupby('item_id')['sales_value'].sum().reset_index()
popular_no_top.sort_values('sales_value', ascending=False, inplace=True)

In [21]:
popular_no_top

,item_id,sales_value
31960,1089093,3784.45
28347,1052294,3658.03
69531,12484608,3019.71
24041,1008288,2828.26
41665,1775642,2673.95
...,...,...
81435,16769888,0.00
11579,880706,0.00
22922,996971,0.00
50096,6396019,0.00


In [22]:
userids = user_item_matrix_top.index.values
itemids = user_item_matrix_top.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [23]:
model = ItemItemRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix_top).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

In [24]:
def get_recomendation(user, model, N=5):
    if user in new_test_users:
        res = popular_no_top.head(N).item_id.tolist()
        
    else:
        res = [id_to_itemid[rec[0]] for rec in model.recommend(userid=userid_to_id[user], 
                                                               user_items=sparse_user_item_top,   # на вход user-item matrix
                                                               N=N, 
                                                               filter_already_liked_items=False, 
                                                               filter_items=None, 
                                                               recalculate_user=True)]
        
    return res

In [25]:
result['itemitem'] = result['user_id'].apply(lambda x: get_recomendation(x, model=model, N=5))

In [27]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)  
    
    precision = flags.sum() / len(recommended_list)    
    
    return precision

In [28]:
result.apply(lambda row: precision_at_k(row['itemitem'], row['actual']), axis=1).mean()

0.15494613124387668

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [29]:
result = pd.read_pickle('predictions_basic.pkl')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[858610, 8019005, 267854, 16219605, 82604]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[2724159, 12262931, 13382060, 6979404, 15798619]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]"


In [30]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    items = np.array(items_weights['item_id'])
    probability = np.array(items_weights['probability'])
    
    recs = np.random.choice(items, size=n, replace=False, p=probability)
    
    return recs.tolist()

In [31]:
items = data_test.groupby('item_id')['quantity'].sum().reset_index()
items['probability'] = items['quantity'] / items['quantity'].sum()
items.head()

,item_id,quantity,probability
0,29512,1,8.828255e-08
1,30356,1,8.828255e-08
2,32392,1,8.828255e-08
3,32439,1,8.828255e-08
4,34873,1,8.828255e-08


Сделайте предсказания

In [32]:
result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items, n=5))
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[858610, 8019005, 267854, 16219605, 82604]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[6534178, 6534166, 6544236, 6533889, 847789]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[2724159, 12262931, 13382060, 6979404, 15798619]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[6534178, 6533889, 1404121, 6534166, 6544236]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [33]:
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[858610, 8019005, 267854, 16219605, 82604]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[6534178, 6534166, 6544236, 6533889, 847789]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[2724159, 12262931, 13382060, 6979404, 15798619]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[6534178, 6533889, 1404121, 6534166, 6544236]"


In [34]:
result.apply(lambda row: precision_at_k(row['weighted_random_recommendation'], row['actual']), axis=1).mean()

0.047698334965720256

In [35]:
result.apply(lambda row: precision_at_k(row['popular_recommendation'], row['actual']), axis=1).mean()

0.15523996082272082

In [36]:
result.apply(lambda row: precision_at_k(row['itemitem'], row['actual']), axis=1).mean()

0.13692458374142857

In [37]:
result.apply(lambda row: precision_at_k(row['cosine'], row['actual']), axis=1).mean()

0.13290891283055686

In [38]:
result.apply(lambda row: precision_at_k(row['tfidf'], row['actual']), axis=1).mean()

0.1389813907933383

### Задание 3. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.
-  *Попробуйте стратегии ансамблирования изученных алгоритмов

Обязательно нужно сделать первые 2 пункта!

In [39]:
def random_recommendation_top(data_train, top=5000, n=5):
    """Случайные рекоммендации"""
    popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
    items = np.array(popularity.sort_values('quantity', ascending=False).head(top).item_id.tolist())    
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [40]:
result['random_recommendation_top'] = result['user_id'].apply(lambda x: random_recommendation_top(data_test, top=5000, n=5))
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,weighted_random_recommendation,random_recommendation_top
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[858610, 8019005, 267854, 16219605, 82604]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[6534178, 6534166, 6544236, 6533889, 847789]","[1085983, 1038985, 934369, 891393, 1048727]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[2724159, 12262931, 13382060, 6979404, 15798619]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[6534178, 6533889, 1404121, 6534166, 6544236]","[1027372, 1103105, 862349, 982755, 1117336]"


In [41]:
result.apply(lambda row: precision_at_k(row['random_recommendation_top'], row['actual']), axis=1).mean()

0.007541625857002928

Остальные алгоритмы и так учитывают наиболее популярные товары

Второй пункт не поняла, что нужно сделать.